In [3]:
"""
Imports Here
"""
"""numpy and torch"""
import numpy as np
import torch

"""PIL"""
from PIL import Image

"""torchvision and utils"""
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset

"""os"""
import os

/Users/oliver/opt/anaconda3/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/oliver/opt/anaconda3/lib/python3.9/site-packages/torchvision/image.so, 6): Library not loaded: @rpath/libpng16.16.dylib
  Referenced from: /Users/oliver/opt/anaconda3/lib/python3.9/site-packages/torchvision/image.so
  Reason: Incompatible library version: image.so requires version 56.0.0 or later, but libpng16.16.dylib provides version 54.0.0'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [4]:
"""
Loading data from local file
"""

'\nLoading data from local file\n'

In [13]:
"""Assumes images have pixel values in range [0,255]"""
def getImages(trainDIRs, testDIRS):
    """Get image to tensor"""
    transform = transforms.Compose([
        transforms.PILToTensor()
    ])
    """Loading data into arrays"""
    xtrain, xtrain, xtest, ytest = [], [], [], []
    """training data"""
    size = [0, 0]
    for i, DIR in enumerate(trainDIRs):
        px = []
        j = 0
        for filename in sorted(os.listdir(DIR)):
            f = os.path.join(DIR, filename)
            img = Image.open(f)
            tensor = transform(img).float()
            tensor.require_grad = True
            px.append(tensor/255)
            j  = (j+1) % 20
            if j == 0:
                xtrain.append(torch.stack(px))
                px = []
                size[i] += 1
    xtrain = torch.stack(xtrain)
    ytrain = torch.from_numpy(np.concatenate((np.ones(size[0]), np.zeros(size[1])), axis=0))
        
    """testing data"""
    size = [0, 0]
    for i, DIR in enumerate(testDIRs):
        px = []
        j = 0
        for filename in sorted(os.listdir(DIR)):
            f = os.path.join(DIR, filename)
            img = Image.open(f)
            tensor = transform(img).float()
            tensor.require_grad = True
            px.append(tensor/255)
            j = (j+1) % 20
            if j == 0:
                xtest.append(torch.stack(px))
                px = []
                size[i] += 1
    xtest = torch.stack(xtest)
    ytest = torch.from_numpy(np.concatenate((np.ones(size[0]), np.zeros(size[1])), axis=0))
    return xtrain, ytrain, xtest, ytest

In [6]:
"""
Dataloader
"""

'\nDataloader\n'

In [7]:
class DatasetWrapper(Dataset):
    def __init__(self, X, y=None):
        self.X, self.y = X, y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        if self.y is None:
            return self.X[idx]
        else:
            return self.X[idx], self.y[idx]

In [8]:
trainDIRs = ['../../../AD_NC/train/AD/', '../../../AD_NC/train/NC']
testDIRs = ['../../../AD_NC/test/AD/', '../../../AD_NC/test/NC']
xtrain, ytrain, xtest, ytest = getImages(trainDIRs, testDIRs)

In [14]:
print(xtrain.shape)
print(xtest.shape)
print(ytest.shape)

torch.Size([1076, 20, 1, 240, 256])
torch.Size([450, 20, 1, 240, 256])
torch.Size([450])


In [98]:
class ConvLayer2(nn.Module):
    def __init__(self):
        super().__init__()
        #pool
        self.pool = nn.MaxPool2d(kernel_size=3, stride=2)
        self.relu = nn.ReLU()
        #first layer
        self.conv11_x = nn.Conv2d(20, 48, kernel_size=(11,11), stride=(4,4), padding=(0,0))
        self.conv11_y = nn.Conv2d(240, 48, kernel_size=(11,3), stride=(4,1), padding=(0,0))
        self.conv11_z = nn.Conv2d(256, 48, kernel_size=(3,11), stride=(1,4), padding=(0,0))
        #second layer
        self.conv5_x = nn.Conv2d(48, 192, kernel_size=(5,5), stride=(2,2), padding=(0,0))
        self.conv5_y = nn.Conv2d(48, 192, kernel_size=(5,3), stride=(2,1), padding=(0,0))
        self.conv5_z = nn.Conv2d(48, 192, kernel_size=(3,5), stride=(1,2), padding=(0,0))
        #projection
        self.l_x = nn.Linear(30, 32)
        self.l_y = nn.Linear(12, 32)
        self.l_z = nn.Linear(10, 32)

    def forward(self, imgs):
        #input N, C, L, W, H
        #first layer
        x_x = self.relu(self.pool(self.conv11_x(imgs.flatten(1,2))))
        x_y = self.relu(self.pool(self.conv11_y(imgs.permute(0,1,3,4,2).flatten(1,2))))
        x_z = self.relu(self.pool(self.conv11_z(imgs.permute(0,1,4,2,3).flatten(1,2))))
        #second layer
        x_x = self.relu(self.pool(self.conv5_x(x_x)))
        x_y = self.relu(self.pool(self.conv5_y(x_y)))
        x_z = self.relu(self.pool(self.conv5_z(x_z)))
        #projection
        x_x = self.l_x(x_x.flatten(2,3))
        x_y = self.l_y(x_y.flatten(2,3))
        x_z = self.l_z(x_z.flatten(2,3))
        return torch.cat([x_x, x_y, x_z], dim=2)
import time
start = time.time()
conv=ConvLayer2()
print(conv(xtrain[0:16,:].permute(0,2,1,3,4)).shape)
end = time.time()
print(round(end-start, 4))

torch.Size([16, 192, 96])
0.2741


In [40]:
conv_11 = nn.Conv2d(20, 64, kernel_size=(11,11), stride=(4,4), padding=(0,0))
import time
total = 0
for i in range(10):
    start=time.time()
    x = conv_11(xtrain[0:16, :].permute(0,2,1,3,4).flatten(1,2))
    end = time.time()
    total += end-start
print(total)

1.004753589630127


In [12]:
import torch.nn as nn
class ConvLayer(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv11 = nn.Conv3d(1, 64, kernel_size=(3,11,11), stride=(1,4,4), padding=(1,0,0))
        self.firstpool = nn.MaxPool3d(kernel_size=3, stride=2)
        self.conv5 = nn.Conv3d(64, 256, kernel_size=(3,5,5), stride=(1,2,2), padding=(1,0,0))
        self.secondpool = nn.MaxPool3d(kernel_size=3, stride=2)

    def forward(self, imgs):
        x = self.conv11(imgs)
        x = self.firstpool(x)
        x = self.conv5(x)
        x = self.secondpool(x)
        return x
    
conv = ConvLayer()
x = conv(xtrain.permute(0,2,1,3,4))
print(x.shape)

torch.Size([1076, 256, 4, 5, 6])


In [10]:
def trainloader(batchsize=16):
    return DataLoader(DatasetWrapper(xtrain, ytrain), batchsize=batchsize, shuffle=True)

def testloader():
    return DataLoader(DatasetWrapper(xtest, ytest), batchsize=1, shuffle=True)

In [11]:
def trainshape():
    return xtrain.shape

def testshape():
    return xtest.shape